In [128]:
from typing import List
from torchvision import transforms, models, datasets
import torch 
from torch.utils.data import DataLoader, Subset, random_split, ConcatDataset
import numpy as np 
import random 
from os.path import exists
import os 
from typing import List, Dict, Tuple
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import accuracy_score, confusion_matrix
import torch.nn as nn
from torch.functional import F
from tqdm import trange
import copy

In [129]:
def get_oxford_splits(
    batch_size: int,
    data_loader_seed: int = 111, 
    pin_memory: bool = False,
    num_workers: int = 1,
    ): 
    K = 5
    num_support = 80
    num_query = 20 

    def seed_worker(worker_id):
        # worker_seed = torch.initial_seed() % 2 ** 32
        np.random.seed(data_loader_seed)
        random.seed(data_loader_seed)
    g = torch.Generator()
    g.manual_seed(data_loader_seed)

    support_classes = list(np.arange(num_support))
    query_classes = list(np.arange(num_query) + num_support)

    img_dim = 64

    train_transforms = transforms.Compose([
        transforms.Resize((img_dim, img_dim)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
    test_transforms = transforms.Compose([
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
    validation_transforms = transforms.Compose([
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
    
    data_path = f'/home/hesam/projects/Phase4/dataset'

    train_ds_full = datasets.Flowers102(root=data_path, split="train", download=True, transform=train_transforms)
    val_ds_full = datasets.Flowers102(root=data_path, split="val", download=True, transform=validation_transforms)
    test_ds_full = datasets.Flowers102(root=data_path, split="test", download=True, transform=test_transforms)

    train_indxs_support = torch.where(torch.isin(torch.tensor(train_ds_full._labels), torch.asarray(support_classes)))[0]
    val_indxs_support = torch.where(torch.isin(torch.tensor(val_ds_full._labels), torch.asarray(support_classes)))[0]
    test_indxs_support = torch.where(torch.isin(torch.tensor(test_ds_full._labels), torch.asarray(support_classes)))[0]
    
    train_ds_subset_support = torch.utils.data.Subset(train_ds_full, train_indxs_support)
    val_ds_subset_support = torch.utils.data.Subset(val_ds_full, val_indxs_support)
    test_ds_subset_support = torch.utils.data.Subset(test_ds_full, test_indxs_support)

    merged_dataset = ConcatDataset([train_ds_subset_support, val_ds_subset_support, test_ds_subset_support])
    ### A, B
    train_ds_support, test_ds_support = torch.utils.data.random_split(merged_dataset, [0.75, 0.25], generator=torch.Generator().manual_seed(42))
    ### 

    train_indxs_query = torch.where(torch.isin(torch.tensor(train_ds_full._labels), torch.asarray(query_classes)))[0]
    N = 10 
    starting_indices = np.arange(0, len(train_indxs_query), N)
    train_indxs_query = np.hstack([train_indxs_query[i:i+K] for i in starting_indices if i + K <= len(train_indxs_query)])
    ### C
    train_ds_query = torch.utils.data.Subset(train_ds_full, train_indxs_query) 
    ###

    val_indxs_query = torch.where(torch.isin(torch.tensor(val_ds_full._labels), torch.asarray(query_classes)))[0]
    test_indxs_query = torch.where(torch.isin(torch.tensor(test_ds_full._labels), torch.asarray(query_classes)))[0]
    val_ds_subset_query = torch.utils.data.Subset(val_ds_full, val_indxs_query)
    test_ds_subset_query = torch.utils.data.Subset(test_ds_full, test_indxs_query)
    
    test_ds_query_full = ConcatDataset([val_ds_subset_query, test_ds_subset_query])
    ### D 
    _, test_ds_query = torch.utils.data.random_split(test_ds_query_full, [0.7, 0.3], generator=torch.Generator().manual_seed(42))
    ###
    
    ### E 
    test_all = ConcatDataset([test_ds_query, test_ds_support])


    A_train_dl = DataLoader(
        train_ds_support,
        batch_size = batch_size, 
        shuffle=True, 
        worker_init_fn=seed_worker,
        generator=g,
        drop_last=False,
        pin_memory=pin_memory,
        num_workers=num_workers
    )
    A_test_dl = DataLoader(
        test_ds_support,
        batch_size = batch_size, 
        shuffle=True, 
        worker_init_fn=seed_worker,
        generator=g,
        drop_last=False,
        pin_memory=pin_memory,
        num_workers=num_workers
    )

    B_train_dl = DataLoader(
        train_ds_query,
        batch_size = batch_size, 
        shuffle=True, 
        worker_init_fn=seed_worker,
        generator=g,
        drop_last=False,
        pin_memory=pin_memory,
        num_workers=num_workers
    )
    B_test_dl = DataLoader(
        test_ds_query,
        batch_size = batch_size, 
        shuffle=True, 
        worker_init_fn=seed_worker,
        generator=g,
        drop_last=False,
        pin_memory=pin_memory,
        num_workers=num_workers
    )
    test_all = DataLoader(
        test_all,
        batch_size = batch_size, 
        shuffle=True, 
        worker_init_fn=seed_worker,
        generator=g,
        drop_last=False,
        pin_memory=pin_memory,
        num_workers=num_workers
    )
    
    return A_train_dl, A_test_dl, B_train_dl, B_test_dl, test_all

In [130]:
def make_dir(dir_name: str):
    """
    creates directory "dir_name" if it doesn't exists
    """
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

In [131]:
def plot_conf(labels, preds, title, dir_, name):
    """
    labels: an [N, ] array containing true labels for N samples
    preds: an [N, ] array containing predications for N samples
    
    saves confusion matrix plot of the given prediction and true labels in 'dir_/name.jpg' 
    """

    conf = confusion_matrix(labels, preds)
    # print("count", torch.unique(preds, return_counts=True))
    plt.clf()
    cm = conf.astype('float') / conf.sum(axis=1)[:, np.newaxis]
    cmap = sns.light_palette("navy", as_cmap=True)
    plt.figure(figsize=(20, 20))
    sns.heatmap(cm, annot=False, cmap=cmap, fmt=".2f", cbar=False)
    # plt.title(f'{title}')
    # plt.xlabel('Predicted Label')
    # plt.ylabel('True Label')
    make_dir(dir_)
    plt.savefig(f'{dir_}/{name}')

In [132]:
A_train_dl, A_test_dl, B_train_dl, B_test_dl, test_all = get_oxford_splits(128)

In [133]:
use_gpu = True
device = torch.device("cuda:0" if torch.cuda.is_available() and use_gpu else "cpu") # New


#### define network with layers

In [134]:
class My_Model(nn.Module):
    
    def __init__(self) -> None:
        super().__init__()
        
        self.flatten = nn.Flatten()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3, 3), padding=1, stride=1)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, stride=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, stride=1)
        self.bn2_1 = nn.BatchNorm2d(64)
        
        self.conv2_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, stride=1)
        self.bn2_2 = nn.BatchNorm2d(64)
        
        self.conv2_3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, stride=1)
        self.bn2_3 = nn.BatchNorm2d(64)
        
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2))
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=96, kernel_size=(3, 3), padding=1, stride=1)
        self.bn3 = nn.BatchNorm2d(96)
        
        self.conv3_1 = nn.Conv2d(in_channels=96, out_channels=96, kernel_size=(3, 3), padding=1, stride=1)
        self.bn3_1 = nn.BatchNorm2d(96)
        
        self.conv3_2 = nn.Conv2d(in_channels=96, out_channels=96, kernel_size=(3, 3), padding=1, stride=1)
        self.bn3_2 = nn.BatchNorm2d(96)
        
        self.conv3_3 = nn.Conv2d(in_channels=96, out_channels=96, kernel_size=(3, 3), padding=1, stride=1)
        self.bn3_3 = nn.BatchNorm2d(96)
        
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2))
        
        self.conv4 = nn.Conv2d(in_channels=96, out_channels=128, kernel_size=(3, 3), padding=1, stride=1)
        self.bn4 = nn.BatchNorm2d(128)
        
        self.conv4_1 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1, stride=1)
        self.bn4_1 = nn.BatchNorm2d(128)
        
        self.conv4_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1, stride=1)
        self.bn4_2 = nn.BatchNorm2d(128)
        
        self.conv4_3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1, stride=1)
        self.bn4_3 = nn.BatchNorm2d(128)        
        
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2))

        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1, stride=1)
        self.bn5 = nn.BatchNorm2d(256)
        
        self.conv5_1 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), padding=1, stride=1)
        self.bn5_1 = nn.BatchNorm2d(256)
        
        self.conv5_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), padding=1, stride=1)
        self.bn5_2 = nn.BatchNorm2d(256)
        
        self.conv5_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), padding=1, stride=1)
        self.bn5_3 = nn.BatchNorm2d(256)


        self.pool4 = nn.AvgPool2d(kernel_size=(2, 2))
        
        self.fc1 = nn.Linear(in_features=4096, out_features=80)
        
    def forward(self, inputs, debug=False):
        x1 = self.conv1(inputs)
        x1 = self.bn1(x1)
        x1 = F.relu(x1)
        
        x2 = self.conv2(x1)
        x2 = self.bn2(x2)
        x2 = F.relu(x2)
        
        x2 = self.conv2_1(x2)
        x2 = self.bn2_1(x2)
        x2 = F.relu(x2)
        
        x2 = self.conv2_2(x2)
        x2 = self.bn2_2(x2)
        x2 = F.relu(x2)
        
        x2 = self.conv2_3(x2)
        x2 = self.bn2_3(x2)
        x2 = F.relu(x2)  

        x2 = self.pool1(x2)

        x3 = self.conv3(x2)
        x3 = self.bn3(x3)
        x3 = F.relu(x3)
        
        x3 = self.conv3_1(x3)
        x3 = self.bn3_1(x3)
        x3 = F.relu(x3)
        
        x3 = self.conv3_2(x3)
        x3 = self.bn3_2(x3)
        x3 = F.relu(x3)
        
        x3 = self.conv3_3(x3)
        x3 = self.bn3_3(x3)
        x3 = F.relu(x3)
    

        x3 = self.pool2(x3)

        x4 = self.conv4(x3)
        x4 = self.bn4(x4)
        x4 = F.relu(x4)
        
        x4 = self.conv4_1(x4)
        x4 = self.bn4_1(x4)
        x4 = F.relu(x4)
        
        x4 = self.conv4_2(x4)
        x4 = self.bn4_2(x4)
        x4 = F.relu(x4)
        
        x4 = self.conv4_3(x4)
        x4 = self.bn4_3(x4)
        x4 = F.relu(x4)

        x4 = self.pool3(x4)

        x5 = self.conv5(x4)
        x5 = self.bn5(x5)
        x5 = F.relu(x5)
        
        x5 = self.conv5_1(x5)
        x5 = self.bn5_1(x5)
        x5 = F.relu(x5)
        
        x5 = self.conv5_2(x5)
        x5 = self.bn5_2(x5)
        x5 = F.relu(x5)
        
        x5 = self.conv5_3(x5)
        x5 = self.bn5_3(x5)
        x5 = F.relu(x5)
  

        x6 = self.pool4(x5)
        x0 = self.flatten(x6)
        outputs = self.fc1(x0)
        
        if debug:
                    print('inputs shape: ', inputs.shape) # inputs in shape [N, C, H, W]
                    print('after flattening: ', x0.shape)
                    print('Activations after 1st fully connected layer: ', x1.shape)
                    print('Activations after 2nd fully connected layer: ', x2.shape)
                    print('Output shape: ', outputs.shape)

        return outputs
       

In [137]:
def train_one_epoch(modelIN: nn.Module, optim: torch.optim.Optimizer,
         dataloader: DataLoader, loss_fn, freeze_fc=False):

    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    running_corrects = 0
    running_loss = 0.0
    modelIN.train() #
    for batch_indx, (inputs, targets) in enumerate(dataloader): # Get a batch of Data
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        outputs = modelIN(inputs) # Forward Pass, [N, 10]
        loss = loss_fn(outputs, targets) # Compute Loss

        loss.backward() # Compute Gradients
        if freeze_fc:
            for param in modelIN.fc1.parameters():
                param.grad[:80] = 0 
            
        optim.step() # Update parameters
        optim.zero_grad() # zero the parameter's 

        _, preds = torch.max(outputs, dim=1) # Explain, [N]
        # print("pred", preds, targets)
        running_corrects += torch.sum(preds == targets)
        running_loss += loss.item()

    epoch_acc = (running_corrects / num_samples) * 100
    epoch_loss = (running_loss / num_batches)

    return epoch_acc, epoch_loss

In [138]:
count = 0

In [139]:
def test_model(modelIN: nn.Module,
         dataloader: DataLoader, loss_fn, dir, cpu=False, cm=False):

    # utils
    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    running_corrects = 0
    running_loss = 0.0
    predictions = []
    labels = []
    modelIN.eval() # you must call `model.eval()` to set dropout and batch normalization layers to evaluation mode before running inference.
    with torch.no_grad(): # explain
        # more on torch.no_grad(): https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html#disabling-gradient-tracking

        for batch_indx, (inputs, targets) in enumerate(dataloader): # Get a batch of Data
            if cpu:
                inputs = inputs.to("cpu")
                targets = targets.to("cpu")
            else:
                inputs = inputs.to(device)
                targets = targets.to(device)
            
            outputs = modelIN(inputs) # Forward Pass
            loss = loss_fn(outputs, targets) # Compute Loss

            # loss.backward() # Compute Gradients
            # optim.step() # Update parameters
            # optim.zero_grad() # zero the parameter's gradients

            _, preds = torch.max(outputs, 1) #
            running_corrects += torch.sum(preds == targets)
            running_loss += loss.item()
            predictions.append(preds.cpu())
            labels.append(targets.cpu())
    # cm flag to plot the cofussion matrix
    if cm:
        global count
        count += 1
        predictions = np.concatenate(predictions, axis=0).flatten()
        labels = np.concatenate(labels, axis=0).flatten()
        plot_conf(labels, predictions, title="CM", dir_=dir, name=f"confusion_matrix-{count}")
    test_acc = (running_corrects / num_samples) * 100
    test_loss = (running_loss / num_batches)

    return test_acc, test_loss

In [140]:
def custom_plot_training_stats(
        acc_hist,
        loss_hist,
        phase_list,
        title: str,
        dir: str,
        index : int,
        name: str = 'acc_loss'
        ):
    fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2, figsize=[14, 6], dpi=100)

    for phase in phase_list:

        lowest_loss_x = np.argmin(np.array(loss_hist[phase]))
        lowest_loss_y = loss_hist[phase][lowest_loss_x]

        ax1.annotate("{:.4f}".format(lowest_loss_y), [lowest_loss_x, lowest_loss_y])
        ax1.plot(loss_hist[phase], '-x', label=f'{phase} loss', markevery = [lowest_loss_x])

        ax1.set_xlabel(xlabel='epochs')
        ax1.set_ylabel(ylabel='loss')

        ax1.grid(color = 'green', linestyle = '--', linewidth = 0.5, alpha=0.75)
        ax1.legend()
        ax1.label_outer()

    # acc:
    for phase in phase_list:
        highest_acc_x = np.argmax(np.array(acc_hist[phase]))
        highest_acc_y = acc_hist[phase][highest_acc_x]

        ax2.annotate("{:.4f}".format(highest_acc_y), [highest_acc_x, highest_acc_y])
        ax2.plot(acc_hist[phase], '-x', label=f'{phase} acc', markevery = [highest_acc_x])

        ax2.set_xlabel(xlabel='epochs')
        ax2.set_ylabel(ylabel='acc')

        ax2.grid(color = 'green', linestyle = '--', linewidth = 0.5, alpha=0.75)
        ax2.legend()
        #ax2.label_outer()

    fig.suptitle(f'{title}')

    plt.savefig(f'chart/{name}-{index}.jpg')
    plt.clf()

In [141]:
def run(train_data, test_data, input_model, index, epochs, lr, test=False, freeze_fc=False, cpu=False, cm=False, dir=None):
    batch_size = 32
    num_epochs = epochs
    learning_rate = lr
    
    model = input_model
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    cross_entropy = nn.CrossEntropyLoss()

    acc_history = {'train': [], 'test': []}
    loss_history = {'train': [], 'test': []}

    for epoch in trange(num_epochs):
        if not test:
            train_acc, train_loss = train_one_epoch(modelIN=model, optim=optimizer, dataloader=train_data, loss_fn=cross_entropy, freeze_fc=freeze_fc)
            print("train: ", train_acc.cpu(), train_loss)
            acc_history['train'].append(train_acc.cpu())
            loss_history['train'].append(train_loss)
        test_acc, test_loss = test_model(modelIN=model, dataloader=test_data, loss_fn=cross_entropy, dir=dir, cpu=cpu, cm=cm)
        print("test: ", test_acc.cpu(), test_loss)
        acc_history['test'].append(test_acc.cpu())
        loss_history['test'].append(test_loss)
        
    custom_plot_training_stats(acc_history, loss_history, ['train', 'test'], title='demp', dir='demo_plots', index=index)


#### train the original model

In [142]:
my_model = My_Model()
run(A_train_dl, A_test_dl, my_model, 1, 90, 0.0001)

  0%|          | 0/90 [00:00<?, ?it/s]

train:  tensor(13.6669) 3.7138580438253044


  1%|          | 1/90 [00:14<21:04, 14.20s/it]

test:  tensor(7.2172) 4.1654169376079855
train:  tensor(31.1241) 2.777406273661433


  2%|▏         | 2/90 [00:27<19:37, 13.38s/it]

...

 99%|█████████▉| 89/90 [19:37<00:12, 12.56s/it]

test:  tensor(78.6086) 1.0168933547460115
train:  tensor(99.9567) 0.00510461103938822


100%|██████████| 90/90 [19:50<00:00, 13.23s/it]

test:  tensor(77.2432) 0.8935288511789762


<Figure size 1400x600 with 0 Axes>

##### check the weight of model before training

In [144]:
my_model.conv2_1.weight

Parameter containing:
tensor([[[[-4.1152e-02,  2.7552e-02,  2.3232e-02],
          [-5.0019e-02, -3.7431e-02,  2.7063e-02],
          [ 4.6284e-03, -4.1788e-02,  3.4469e-02]],

         [[ 1.0118e-03, -3.1182e-02, -1.6825e-02],
          [-3.5135e-02, -4.1477e-02,  4.8241e-03],
          [ 1.6821e-02, -2.0892e-02, -2.0368e-02]],

         [[ 2.6120e-02, -2.7894e-02, -1.2092e-02],
          [ 1.3028e-02, -3.5441e-02, -1.2177e-02],
          [-3.2591e-02, -2.6870e-02,  3.9030e-02]],

         ...,

         [[ 2.6396e-02,  3.2726e-02,  7.4274e-03],
          [-8.5307e-03, -2.7306e-02, -3.1041e-02],
          [-1.1025e-02, -1.9495e-02,  2.3737e-02]],

         [[-2.6593e-02,  3.9382e-02, -8.3937e-03],
          [-3.1704e-02, -6.0318e-03,  5.3831e-03],
          [-4.7803e-02, -4.3978e-02,  5.3610e-06]],

         [[-3.1691e-02,  1.3603e-03, -2.7139e-03],
          [ 1.4907e-02, -1.9542e-03,  8.6123e-05],
          [-7.5084e-03, -3.1214e-03,  1.3781e-02]]],


        [[[-1.7927e-02,  1.9769

In [145]:
my_model.fc1.weight

Parameter containing:
tensor([[-0.0148, -0.0265, -0.0253,  ...,  0.0032,  0.0118, -0.0070],
        [ 0.0125,  0.0121,  0.0150,  ..., -0.0289,  0.0135,  0.0117],
        [-0.0210, -0.0359, -0.0215,  ...,  0.0106,  0.0034,  0.0037],
        ...,
        [ 0.0109,  0.0057, -0.0067,  ...,  0.0134, -0.0179,  0.0016],
        [ 0.0017, -0.0018, -0.0309,  ..., -0.0008, -0.0080, -0.0014],
        [-0.0366, -0.0081,  0.0072,  ..., -0.0128, -0.0209,  0.0118]],
       device='cuda:0', requires_grad=True)

# Phase 2

## First Way

In [146]:
def transfer_learning(model, freeze=False):
    """
    Perform transfer learning by modifying the given model.

    Args:
    - model: Pretrained neural network model
    - freeze (bool): Flag to freeze the pretrained weights (default is False)

    Returns:
    - new_model: Modified neural network model for transfer learning
    """
    # Deep copy the model to avoid modifying the original
    copyModel = copy.deepcopy(model)
    # Create a new model instance
    new_model = type(copyModel)()
    
    # Iterate over child modules of the copied model
    for child_name, child_module in copyModel.named_children():
        # Exclude certain layers from transfer learning
        if child_name not in ["fc1", "output"]:
            new_model.add_module(child_name, child_module)
            
    # Freeze the pretrained weights if specified
    if freeze:
        for param in new_model.parameters():
            param.requires_grad = False
    
    # Extract dimensions of input and output layers of the original model
    old_input_classes = copyModel.fc1.in_features
    new_fc_len = copyModel.fc1.out_features + 20
    
    # Extract weights and bias from the original fully connected layer
    old_weights = copyModel.fc1.weight.data
    old_bias = copyModel.fc1.bias.data
            
    # Create a new fully connected layer with modified dimensions
    new_fc = nn.Linear(old_input_classes, new_fc_len)
    
    # Copy pretrained weights and bias to the new fully connected layer
    with torch.no_grad():
        new_fc.weight[:80, :old_input_classes].copy_(old_weights)
        new_fc.bias[:80].copy_(old_bias)
        
    # Add the new fully connected layer to the new model
    new_model.add_module("fc1", new_fc)
    
    return new_model

In [147]:
newModel1 = transfer_learning(model=my_model)

#### weights after transfer

In [149]:
newModel1.conv2_1.weight

Parameter containing:
tensor([[[[-4.1152e-02,  2.7552e-02,  2.3232e-02],
          [-5.0019e-02, -3.7431e-02,  2.7063e-02],
          [ 4.6284e-03, -4.1788e-02,  3.4469e-02]],

         [[ 1.0118e-03, -3.1182e-02, -1.6825e-02],
          [-3.5135e-02, -4.1477e-02,  4.8241e-03],
          [ 1.6821e-02, -2.0892e-02, -2.0368e-02]],

         [[ 2.6120e-02, -2.7894e-02, -1.2092e-02],
          [ 1.3028e-02, -3.5441e-02, -1.2177e-02],
          [-3.2591e-02, -2.6870e-02,  3.9030e-02]],

         ...,

         [[ 2.6396e-02,  3.2726e-02,  7.4274e-03],
          [-8.5307e-03, -2.7306e-02, -3.1041e-02],
          [-1.1025e-02, -1.9495e-02,  2.3737e-02]],

         [[-2.6593e-02,  3.9382e-02, -8.3937e-03],
          [-3.1704e-02, -6.0318e-03,  5.3831e-03],
          [-4.7803e-02, -4.3978e-02,  5.3610e-06]],

         [[-3.1691e-02,  1.3603e-03, -2.7139e-03],
          [ 1.4907e-02, -1.9542e-03,  8.6123e-05],
          [-7.5084e-03, -3.1214e-03,  1.3781e-02]]],


        [[[-1.7927e-02,  1.9769

In [59]:
newModel1.fc1.weight

Parameter containing:
tensor([[-0.0170, -0.0145, -0.0249,  ..., -0.0205, -0.0112, -0.0009],
        [-0.0257,  0.0155, -0.0224,  ...,  0.0049, -0.0210, -0.0069],
        [-0.0129,  0.0178, -0.0125,  ..., -0.0094, -0.0255, -0.0131],
        ...,
        [ 0.0121,  0.0036, -0.0042,  ...,  0.0038, -0.0079,  0.0064],
        [-0.0148,  0.0133,  0.0133,  ...,  0.0150,  0.0030, -0.0145],
        [-0.0070,  0.0070, -0.0134,  ...,  0.0072, -0.0007,  0.0016]],
       requires_grad=True)

### Train on B

In [150]:
run(train_data=B_train_dl, test_data=B_test_dl, input_model=newModel1, index=2, epochs=20, lr=0.0001, test=False)

  0%|          | 0/20 [00:00<?, ?it/s]

train:  tensor(3.) 3.7360117435455322


  5%|▌         | 1/20 [00:01<00:37,  1.95s/it]

test:  tensor(3.8610) 3.3133307456970216
train:  tensor(17.) 3.0067801475524902


 10%|█         | 2/20 [00:03<00:32,  1.81s/it]

...

 95%|█████████▌| 19/20 [00:32<00:01,  1.69s/it]

test:  tensor(43.4363) 1.9730603456497193
train:  tensor(100.) 0.16455689072608948


100%|██████████| 20/20 [00:34<00:00,  1.70s/it]

test:  tensor(43.2432) 1.9940651416778565


<Figure size 1400x600 with 0 Axes>

#### weights after train 

In [151]:
newModel1.conv2_1.weight

Parameter containing:
tensor([[[[-0.0412,  0.0273,  0.0228],
          [-0.0505, -0.0379,  0.0266],
          [ 0.0041, -0.0421,  0.0342]],

         [[ 0.0009, -0.0311, -0.0161],
          [-0.0348, -0.0408,  0.0056],
          [ 0.0172, -0.0198, -0.0195]],

         [[ 0.0264, -0.0272, -0.0119],
          [ 0.0135, -0.0348, -0.0117],
          [-0.0320, -0.0265,  0.0389]],

         ...,

         [[ 0.0268,  0.0332,  0.0080],
          [-0.0082, -0.0270, -0.0306],
          [-0.0108, -0.0192,  0.0241]],

         [[-0.0262,  0.0391, -0.0088],
          [-0.0318, -0.0060,  0.0052],
          [-0.0477, -0.0442, -0.0002]],

         [[-0.0310,  0.0020, -0.0021],
          [ 0.0155, -0.0014,  0.0006],
          [-0.0070, -0.0027,  0.0142]]],


        [[[-0.0170,  0.0210,  0.0433],
          [-0.0281,  0.0368,  0.0209],
          [ 0.0300,  0.0135,  0.0324]],

         [[-0.0160,  0.0275, -0.0373],
          [-0.0280, -0.0265, -0.0431],
          [ 0.0223, -0.0215, -0.0303]],

         

###

In [152]:
newModel1.fc1.weight

Parameter containing:
tensor([[-0.0158, -0.0277, -0.0263,  ...,  0.0020,  0.0107, -0.0081],
        [ 0.0110,  0.0106,  0.0136,  ..., -0.0302,  0.0122,  0.0104],
        [-0.0224, -0.0372, -0.0229,  ...,  0.0094,  0.0021,  0.0028],
        ...,
        [-0.0052, -0.0069,  0.0171,  ...,  0.0010,  0.0077,  0.0102],
        [ 0.0084, -0.0062, -0.0088,  ..., -0.0137, -0.0104,  0.0009],
        [ 0.0014, -0.0123,  0.0134,  ..., -0.0027, -0.0044,  0.0150]],
       device='cuda:0', requires_grad=True)

#### test on A test after train on B

In [75]:
nm = newModel1.to("cpu")
acc, _ = test_model(nm, A_test_dl, nn.CrossEntropyLoss(), dir="Conf_B1_after_train_A-test", cpu=True)
print(acc)

tensor(9.0377)


#### for test all and plot test curve

In [ ]:
run(B_train_dl , test_all, newModel1, index=3, epochs=20, lr=0.0001, cm=True, dir="CM1")

## Second Way

In [153]:
new_model2 = transfer_learning(my_model, freeze=True)

#### weight after transfer

In [154]:
new_model2.conv2_1.weight

Parameter containing:
tensor([[[[-4.1152e-02,  2.7552e-02,  2.3232e-02],
          [-5.0019e-02, -3.7431e-02,  2.7063e-02],
          [ 4.6284e-03, -4.1788e-02,  3.4469e-02]],

         [[ 1.0118e-03, -3.1182e-02, -1.6825e-02],
          [-3.5135e-02, -4.1477e-02,  4.8241e-03],
          [ 1.6821e-02, -2.0892e-02, -2.0368e-02]],

         [[ 2.6120e-02, -2.7894e-02, -1.2092e-02],
          [ 1.3028e-02, -3.5441e-02, -1.2177e-02],
          [-3.2591e-02, -2.6870e-02,  3.9030e-02]],

         ...,

         [[ 2.6396e-02,  3.2726e-02,  7.4274e-03],
          [-8.5307e-03, -2.7306e-02, -3.1041e-02],
          [-1.1025e-02, -1.9495e-02,  2.3737e-02]],

         [[-2.6593e-02,  3.9382e-02, -8.3937e-03],
          [-3.1704e-02, -6.0318e-03,  5.3831e-03],
          [-4.7803e-02, -4.3978e-02,  5.3610e-06]],

         [[-3.1691e-02,  1.3603e-03, -2.7139e-03],
          [ 1.4907e-02, -1.9542e-03,  8.6123e-05],
          [-7.5084e-03, -3.1214e-03,  1.3781e-02]]],


        [[[-1.7927e-02,  1.9769

In [155]:
new_model2.fc1.weight

Parameter containing:
tensor([[-0.0148, -0.0265, -0.0253,  ...,  0.0032,  0.0118, -0.0070],
        [ 0.0125,  0.0121,  0.0150,  ..., -0.0289,  0.0135,  0.0117],
        [-0.0210, -0.0359, -0.0215,  ...,  0.0106,  0.0034,  0.0037],
        ...,
        [-0.0057,  0.0136,  0.0093,  ...,  0.0092,  0.0100, -0.0023],
        [ 0.0122,  0.0152,  0.0062,  ...,  0.0132,  0.0029,  0.0114],
        [-0.0082,  0.0127,  0.0095,  ..., -0.0137,  0.0123, -0.0109]],
       requires_grad=True)

### Train on B

In [156]:
run(B_train_dl, B_test_dl, new_model2, index=4, epochs=40, lr=0.0001)

  0%|          | 0/40 [00:00<?, ?it/s]

train:  tensor(2.) 3.7205376625061035


  2%|▎         | 1/40 [00:01<01:05,  1.68s/it]

test:  tensor(2.8958) 3.452285575866699
train:  tensor(6.) 3.5804147720336914


  5%|▌         | 2/40 [00:03<01:00,  1.60s/it]

...

 98%|█████████▊| 39/40 [01:00<00:01,  1.56s/it]

test:  tensor(39.1892) 2.230818843841553
train:  tensor(97.) 0.9560943841934204


100%|██████████| 40/40 [01:02<00:00,  1.56s/it]

test:  tensor(40.3475) 2.117531442642212


<Figure size 1400x600 with 0 Axes>

#### weights after train 

In [157]:
new_model2.conv2_1.weight

Parameter containing:
tensor([[[[-4.1152e-02,  2.7552e-02,  2.3232e-02],
          [-5.0019e-02, -3.7431e-02,  2.7063e-02],
          [ 4.6284e-03, -4.1788e-02,  3.4469e-02]],

         [[ 1.0118e-03, -3.1182e-02, -1.6825e-02],
          [-3.5135e-02, -4.1477e-02,  4.8241e-03],
          [ 1.6821e-02, -2.0892e-02, -2.0368e-02]],

         [[ 2.6120e-02, -2.7894e-02, -1.2092e-02],
          [ 1.3028e-02, -3.5441e-02, -1.2177e-02],
          [-3.2591e-02, -2.6870e-02,  3.9030e-02]],

         ...,

         [[ 2.6396e-02,  3.2726e-02,  7.4274e-03],
          [-8.5307e-03, -2.7306e-02, -3.1041e-02],
          [-1.1025e-02, -1.9495e-02,  2.3737e-02]],

         [[-2.6593e-02,  3.9382e-02, -8.3937e-03],
          [-3.1704e-02, -6.0318e-03,  5.3831e-03],
          [-4.7803e-02, -4.3978e-02,  5.3610e-06]],

         [[-3.1691e-02,  1.3603e-03, -2.7139e-03],
          [ 1.4907e-02, -1.9542e-03,  8.6123e-05],
          [-7.5084e-03, -3.1214e-03,  1.3781e-02]]],


        [[[-1.7927e-02,  1.9769

In [158]:
new_model2.fc1.weight

Parameter containing:
tensor([[-0.0170, -0.0284, -0.0274,  ...,  0.0014,  0.0098, -0.0091],
        [ 0.0108,  0.0102,  0.0129,  ..., -0.0309,  0.0114,  0.0098],
        [-0.0230, -0.0378, -0.0232,  ...,  0.0085,  0.0016,  0.0020],
        ...,
        [-0.0024,  0.0171,  0.0120,  ...,  0.0056,  0.0121, -0.0062],
        [ 0.0083,  0.0113,  0.0027,  ...,  0.0114,  0.0043,  0.0089],
        [-0.0056,  0.0159,  0.0126,  ..., -0.0106,  0.0153, -0.0079]],
       device='cuda:0', requires_grad=True)

#### test on A after train on B with freezing all layers except fc

In [159]:
nm1 = new_model2.to("cpu")
epoch_acc, epoch_loss = test_model(nm1, A_test_dl, loss_fn=nn.CrossEntropyLoss(), dir="Conf_B2_after_train_A-test", cpu=True)
print(epoch_acc)

tensor(47.3992)


#### for test all and plot test curve

In [ ]:
run(B_train_dl , test_all, new_model2, index=5, epochs=20, lr=0.0001, cm=True, dir="CM2")

## Third Way

In [160]:
new_model3 = transfer_learning(my_model, freeze=True)

##### weights after transfer

In [161]:
new_model3.conv2_1.weight

Parameter containing:
tensor([[[[-4.1152e-02,  2.7552e-02,  2.3232e-02],
          [-5.0019e-02, -3.7431e-02,  2.7063e-02],
          [ 4.6284e-03, -4.1788e-02,  3.4469e-02]],

         [[ 1.0118e-03, -3.1182e-02, -1.6825e-02],
          [-3.5135e-02, -4.1477e-02,  4.8241e-03],
          [ 1.6821e-02, -2.0892e-02, -2.0368e-02]],

         [[ 2.6120e-02, -2.7894e-02, -1.2092e-02],
          [ 1.3028e-02, -3.5441e-02, -1.2177e-02],
          [-3.2591e-02, -2.6870e-02,  3.9030e-02]],

         ...,

         [[ 2.6396e-02,  3.2726e-02,  7.4274e-03],
          [-8.5307e-03, -2.7306e-02, -3.1041e-02],
          [-1.1025e-02, -1.9495e-02,  2.3737e-02]],

         [[-2.6593e-02,  3.9382e-02, -8.3937e-03],
          [-3.1704e-02, -6.0318e-03,  5.3831e-03],
          [-4.7803e-02, -4.3978e-02,  5.3610e-06]],

         [[-3.1691e-02,  1.3603e-03, -2.7139e-03],
          [ 1.4907e-02, -1.9542e-03,  8.6123e-05],
          [-7.5084e-03, -3.1214e-03,  1.3781e-02]]],


        [[[-1.7927e-02,  1.9769

In [162]:
new_model3.fc1.weight

Parameter containing:
tensor([[-0.0148, -0.0265, -0.0253,  ...,  0.0032,  0.0118, -0.0070],
        [ 0.0125,  0.0121,  0.0150,  ..., -0.0289,  0.0135,  0.0117],
        [-0.0210, -0.0359, -0.0215,  ...,  0.0106,  0.0034,  0.0037],
        ...,
        [-0.0082,  0.0088, -0.0012,  ...,  0.0147, -0.0134,  0.0134],
        [ 0.0123,  0.0021,  0.0028,  ..., -0.0142,  0.0143, -0.0069],
        [ 0.0066,  0.0025, -0.0051,  ..., -0.0004,  0.0048,  0.0136]],
       requires_grad=True)

### train third model 

In [163]:
run(B_train_dl, B_test_dl, new_model3, index=21, epochs=50, lr=0.0001, freeze_fc=True)

  0%|          | 0/50 [00:00<?, ?it/s]

train:  tensor(4.) 3.7633426189422607


  2%|▏         | 1/50 [00:01<01:27,  1.78s/it]

test:  tensor(2.7027) 3.6670021533966066
train:  tensor(5.0000) 3.632528781890869


  4%|▍         | 2/50 [00:03<01:18,  1.65s/it]

...

 98%|█████████▊| 49/50 [01:17<00:01,  1.56s/it]

test:  tensor(35.5212) 2.228701448440552
train:  tensor(88.) 0.9179799556732178


100%|██████████| 50/50 [01:18<00:00,  1.57s/it]

test:  tensor(35.5212) 2.335897159576416


<Figure size 1400x600 with 0 Axes>

##### check the weights after train the model

In [164]:
new_model3.conv2_1.weight

Parameter containing:
tensor([[[[-4.1152e-02,  2.7552e-02,  2.3232e-02],
          [-5.0019e-02, -3.7431e-02,  2.7063e-02],
          [ 4.6284e-03, -4.1788e-02,  3.4469e-02]],

         [[ 1.0118e-03, -3.1182e-02, -1.6825e-02],
          [-3.5135e-02, -4.1477e-02,  4.8241e-03],
          [ 1.6821e-02, -2.0892e-02, -2.0368e-02]],

         [[ 2.6120e-02, -2.7894e-02, -1.2092e-02],
          [ 1.3028e-02, -3.5441e-02, -1.2177e-02],
          [-3.2591e-02, -2.6870e-02,  3.9030e-02]],

         ...,

         [[ 2.6396e-02,  3.2726e-02,  7.4274e-03],
          [-8.5307e-03, -2.7306e-02, -3.1041e-02],
          [-1.1025e-02, -1.9495e-02,  2.3737e-02]],

         [[-2.6593e-02,  3.9382e-02, -8.3937e-03],
          [-3.1704e-02, -6.0318e-03,  5.3831e-03],
          [-4.7803e-02, -4.3978e-02,  5.3610e-06]],

         [[-3.1691e-02,  1.3603e-03, -2.7139e-03],
          [ 1.4907e-02, -1.9542e-03,  8.6123e-05],
          [-7.5084e-03, -3.1214e-03,  1.3781e-02]]],


        [[[-1.7927e-02,  1.9769

In [165]:
new_model3.fc1.weight

Parameter containing:
tensor([[-1.4840e-02, -2.6468e-02, -2.5326e-02,  ...,  3.2259e-03,
          1.1839e-02, -7.0226e-03],
        [ 1.2509e-02,  1.2070e-02,  1.4986e-02,  ..., -2.8877e-02,
          1.3546e-02,  1.1703e-02],
        [-2.1009e-02, -3.5923e-02, -2.1500e-02,  ...,  1.0603e-02,
          3.4309e-03,  3.6793e-03],
        ...,
        [-3.8068e-03,  1.3182e-02,  2.8409e-03,  ...,  1.0496e-02,
         -9.5307e-03,  8.7829e-03],
        [ 7.4267e-03, -2.7958e-03,  6.6207e-05,  ..., -1.3116e-02,
          1.8094e-02, -9.1179e-03],
        [ 1.0495e-02,  6.7059e-03, -1.1667e-03,  ...,  3.6929e-03,
          8.8386e-03,  1.8029e-02]], device='cuda:0', requires_grad=True)

#### for test all and plot test curve

In [ ]:
run(B_train_dl , test_all, new_model3, index=155, epochs=20, lr=0.0001, cm=True, freeze_fc=True, dir="CM3")